In [416]:
# !python -m pip install geopandas
import pandas as pd
import numpy as np
import geopandas as gpd
from scipy.stats import zscore


In [417]:
#reading rheast and extracting cnames
rheas = pd.read_csv(r'data/rheas/zmb_n_25_chirps__newmask_1999_2023_maize.csv')


In [418]:
# rename cname column to district
rheas.rename(columns={'cname':'district'}, inplace=True)

In [419]:
# convert columns to lower and cname names
rheas.columns=rheas.columns.str.lower()
rheas.district=rheas.district.str.lower()
rheas.district=rheas.district.str.replace('-', '', regex=True)
rheas.district=rheas.district.str.replace(' ', '', regex=True)

In [420]:
# clean rheas survey data
print(len(rheas))
#drop duplicate inputs
rheas = rheas.drop_duplicates()
print(len(rheas))
rheas

295779
295779


,id,gid,district,season,ccode,ensemble,planting,harvest,wsgd,gwad
0,1083083,94,nkeyema,1998 1,110011,29,1999-01-01,1999-05-18,0.000,3716
1,1083087,94,nkeyema,1998 1,110011,33,1999-01-01,1999-05-18,0.000,3716
2,1056596,83,mushindano,1998 1,108011,15,1999-01-01,1999-05-18,0.000,3716
3,1085429,95,nsama,1998 1,107009,15,1999-01-01,1999-05-18,0.000,3716
4,929310,30,kalabo,1998 1,110001,41,1999-01-01,1999-05-18,0.000,3716
...,...,...,...,...,...,...,...,...,...,...
295774,1289442,63,mambwe,2022 1,103005,8,2023-01-30,2023-06-04,0.000,4090
295775,1229437,38,kasama,2022 1,107003,3,2023-01-30,2023-06-07,0.182,4447
295776,1229441,38,kasama,2022 1,107003,7,2023-01-30,2023-06-06,0.000,4161
295777,1289479,63,mambwe,2022 1,103005,45,2023-01-30,2023-06-04,0.000,4230


In [421]:
# drop unnecessary columns
rheas.rename(columns={'cname':'district'}, inplace=True)
rheas=rheas.drop(['id','gid', 'ccode', 'ensemble'], axis=1)

In [422]:
# # convert appropriate columns to numeric
# rheas.planted_ha=pd.to_numeric(rheas.planted_ha)
# rheas.harvested_ha=pd.to_numeric(rheas.harvested_ha)
# rheas.production_mt=pd.to_numeric(rheas.production_mt)
# rheas['yield(mt/ha)']=pd.to_numeric(rheas['yield(mt/ha)'])
# convert dates and calculate yield
rheas['planting']=pd.to_datetime(rheas['planting'], format="mixed")
rheas['year'] = rheas['planting'].dt.year
rheas['yield']=rheas['gwad']/1000
rheas['month']=rheas['planting'].dt.month
rheas['month'].unique()
rheas

,district,season,planting,harvest,wsgd,gwad,year,yield,month
0,nkeyema,1998 1,1999-01-01,1999-05-18,0.000,3716,1999,3.716,1
1,nkeyema,1998 1,1999-01-01,1999-05-18,0.000,3716,1999,3.716,1
2,mushindano,1998 1,1999-01-01,1999-05-18,0.000,3716,1999,3.716,1
3,nsama,1998 1,1999-01-01,1999-05-18,0.000,3716,1999,3.716,1
4,kalabo,1998 1,1999-01-01,1999-05-18,0.000,3716,1999,3.716,1
...,...,...,...,...,...,...,...,...,...
295774,mambwe,2022 1,2023-01-30,2023-06-04,0.000,4090,2023,4.090,1
295775,kasama,2022 1,2023-01-30,2023-06-07,0.182,4447,2023,4.447,1
295776,kasama,2022 1,2023-01-30,2023-06-06,0.000,4161,2023,4.161,1
295777,mambwe,2022 1,2023-01-30,2023-06-04,0.000,4230,2023,4.230,1


In [423]:
# derive season column
part = [2, 2, 2, 2,2, 2, 0, 0, 0, 1, 1, 1]
month_to_part=dict(zip(range(1,13), part))
rheas['season2']= rheas['planting'].dt.month.map(month_to_part)
# rheas_chirps_part1[rheas_chirps_part1['season']=='no']
rheas

,district,season,planting,harvest,wsgd,gwad,year,yield,month,season2
0,nkeyema,1998 1,1999-01-01,1999-05-18,0.000,3716,1999,3.716,1,2
1,nkeyema,1998 1,1999-01-01,1999-05-18,0.000,3716,1999,3.716,1,2
2,mushindano,1998 1,1999-01-01,1999-05-18,0.000,3716,1999,3.716,1,2
3,nsama,1998 1,1999-01-01,1999-05-18,0.000,3716,1999,3.716,1,2
4,kalabo,1998 1,1999-01-01,1999-05-18,0.000,3716,1999,3.716,1,2
...,...,...,...,...,...,...,...,...,...,...
295774,mambwe,2022 1,2023-01-30,2023-06-04,0.000,4090,2023,4.090,1,2
295775,kasama,2022 1,2023-01-30,2023-06-07,0.182,4447,2023,4.447,1,2
295776,kasama,2022 1,2023-01-30,2023-06-06,0.000,4161,2023,4.161,1,2
295777,mambwe,2022 1,2023-01-30,2023-06-04,0.000,4230,2023,4.230,1,2


In [424]:
# Extract data only for the seasons
rheas_part1= rheas.loc[rheas['season2']==1]
rheas_part2= rheas.loc[rheas['season2']==2]
# rheas_chirps_part2.rheas_chirps_part2.loc[rheas_chirps_part2['year']==2022]
rheas=pd.concat([rheas_part1, rheas_part2], axis=0, ignore_index=True)



# df=[]
# years=range(2011,2013)
# for year in years:
#     df1=rheas.loc[rheas['year']==year]
    # if df1['season2']!=1:
    #     rheas=rheas_part1.loc[rheas_part2['year']==year+1]
    # else:
    #     df3=rheas_part1.loc[rheas_part1['year']==year]


df=[]
years=range(1999,2024)
for year in years:
    rheas=rheas_part1.loc[rheas_part1['year']==year]
    df3=rheas_part2.loc[rheas_part2['year']==year+1]
    df4=pd.concat([rheas, df3], axis=0, ignore_index=True)
    df4['season_year']=year
    df.append(df4)
    df5=pd.concat(df,axis=0,ignore_index=True)



#     df1=rheas.loc[rheas['year']==year]
#     df2=rheas_part1.loc[rheas_part1['year']==i]
#     # df3=rheas_part1.loc[rheas_part2['year']==i]
#     # df4=pd.concat([df2, df3], axis=0, ignore_index=True)
#     # df4['season_year']=i-1
#     # df.append(df4)
#     # df5=pd.concat(df,axis=1,ignore_index=True)
# # for year in years:
# #     print(year+1)
# df2=rheas_part1.loc[rheas_part1['year']==2013]
df5


,district,season,planting,harvest,wsgd,gwad,year,yield,month,season2,season_year
0,kalabo,1999 1,1999-10-15,2000-02-18,0.000,4680,1999,4.680,10,1,1999
1,zambezi,1999 1,1999-10-15,2000-02-18,0.000,4680,1999,4.680,10,1,1999
2,nkeyema,1999 1,1999-10-15,2000-02-18,0.000,4680,1999,4.680,10,1,1999
3,mulobezi,1999 1,1999-10-15,2000-02-18,0.000,4680,1999,4.680,10,1,1999
4,mushindano,1999 1,1999-10-15,2000-02-18,0.000,4680,1999,4.680,10,1,1999
...,...,...,...,...,...,...,...,...,...,...,...
284174,mambwe,2022 1,2023-01-30,2023-06-04,0.000,4090,2023,4.090,1,2,2022
284175,kasama,2022 1,2023-01-30,2023-06-07,0.182,4447,2023,4.447,1,2,2022
284176,kasama,2022 1,2023-01-30,2023-06-06,0.000,4161,2023,4.161,1,2,2022
284177,mambwe,2022 1,2023-01-30,2023-06-04,0.000,4230,2023,4.230,1,2,2022


In [425]:

#calculate yield
df5 = df5.groupby(['district','season_year'],as_index=False).agg({'yield':'mean'})
# rheas['yield'] = rheas['production_mt'] / rheas['area_ha']
districts=list(df5.district.unique())


In [426]:
# #calculate z scores of yield and flag any anomalies
df6 = []
# rheas = list(rheas.reset_index())
for d in districts:
    df7 = df5.loc[df5['district']==d].copy()
    df7['zyield'] = zscore(df7['yield'])
    df6.append(df7)
    df8=pd.concat(df6,axis=0, ignore_index=True)
# rheas.loc[rheas['district']=='kalomo']
# rheas.loc[rheas['district']=='zambezi']
df8

,district,season_year,yield,zyield
0,chadiza,1999,4.261113,1.502019
1,chadiza,2000,4.168920,0.965961
2,chadiza,2001,4.147630,0.842171
3,chadiza,2002,4.040450,0.218973
4,chadiza,2003,4.237460,1.364487
...,...,...,...,...
2779,zimba,2018,3.656320,-1.990595
2780,zimba,2019,3.926810,-0.441294
2781,zimba,2020,4.151760,0.847165
2782,zimba,2021,3.899850,-0.595715


In [427]:
# add a flag to the dataframe
df8['flag'] = 0
df8['flag'].loc[(df8['zyield']>1.96)|(df8['zyield']<-1.96)]= 1
df8['flag'].loc[(df8['zyield']>2.58)|((df8['zyield']<-2.58))] = 2
df8

C:\Users\kasit\AppData\Local\Temp\ipykernel_2752\1792788726.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df8['flag'].loc[(df8['zyield']>1.96)|(df8['zyield']<-1.96)]= 1
C:\Users\kasit\AppData\Local\Temp\ipykernel_2752\1792788726.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df8['flag'].loc[(df8['zyield']>2.58)|((df8['zyield']<-2.58))] = 2


,district,season_year,yield,zyield,flag
0,chadiza,1999,4.261113,1.502019,0
1,chadiza,2000,4.168920,0.965961,0
2,chadiza,2001,4.147630,0.842171,0
3,chadiza,2002,4.040450,0.218973,0
4,chadiza,2003,4.237460,1.364487,0
...,...,...,...,...,...
2779,zimba,2018,3.656320,-1.990595,1
2780,zimba,2019,3.926810,-0.441294,0
2781,zimba,2020,4.151760,0.847165,0
2782,zimba,2021,3.899850,-0.595715,0


In [428]:
#rename season-year to year
# rheas.rename(columns={'season_year'})

In [429]:
# remove outliers from the data i.e weedout flag=2


In [430]:
# save to csv
df8.to_csv('//data/rheas/zambia_rheas_data_clean.csv')